In [2]:
import pandas as pd
import sys

sys.path.insert(0, "/Users/hkhivrenko/src-2/stock-valuation")

In [19]:
def get_financials():
    return (
        pd        
            .read_csv("uber_tenk_financials.csv", index_col=0)
            .rename_axis(None)
            .fillna(0)
            .astype(int)
            .sort_index(axis=1)
    )

In [20]:
financials = get_financials()

In [21]:
financials.head()

,2020,2021,2022,2023,2024
us-gaap_CashAndCashEquivalentsAtCarryingValue,5647000000,4295000000,4208000000,4680000000,5893000000
us-gaap_MarketableSecuritiesCurrent,1180000000,0,103000000,727000000,1084000000
us-gaap_RestrictedCashAndCashEquivalentsAtCarryingValue,250000000,631000000,680000000,805000000,545000000
us-gaap_AccountsReceivableNetCurrent,1073000000,2439000000,2779000000,3404000000,3333000000
us-gaap_PrepaidExpenseAndOtherAssetsCurrent,1215000000,1454000000,1479000000,1681000000,1390000000


In [22]:
from stock_valuation.financials_mappers.composite_financials_mapper import CompositeFinancialsMapper
from stock_valuation.financials_mappers.row_financials_mapper import RowFinancialsMapper
from stock_valuation.financials_mappers.delta_financials_mapper import DeltaFinancialsMapper
from stock_valuation.constants.financial_keys import REVENUE
from stock_valuation.constants.financial_keys import COST_OF_REVENUE
from stock_valuation.constants.financial_keys import OPERATING_EXPENSE
from stock_valuation.constants.financial_keys import DEPRECIATION
from stock_valuation.constants.financial_keys import PROVISION_FOR_TAXES
from stock_valuation.constants.financial_keys import CAPITAL_EXPENDITURE
from stock_valuation.constants.financial_keys import WORKING_CAPITAL_DELTA
from stock_valuation.constants.us_gaap_financial_keys import US_GAAP_REVENUES
from stock_valuation.constants.us_gaap_financial_keys import US_GAAP_REVENUE_FROM_CONTRACT_WITH_CUSTOMER_EXCLUDING_ASSESSED_TAX
from stock_valuation.constants.us_gaap_financial_keys import US_GAAP_COST_OF_REVENUE
from stock_valuation.constants.us_gaap_financial_keys import US_GAAP_COST_OF_GOODS_AND_SERVICE_EXCLUDING_DEPRECIATION_DEPLETION_AND_AMORTIZATION
from stock_valuation.constants.us_gaap_financial_keys import US_GAAP_OPERATING_INCOME_LOSS
from stock_valuation.constants.us_gaap_financial_keys import US_GAAP_DEPRECIATION_DEPLETION_AND_AMORTIZATION
from stock_valuation.constants.us_gaap_financial_keys import US_GAAP_INCOME_TAX_EXPENSE_BENEFIT
from stock_valuation.constants.us_gaap_financial_keys import US_GAAP_PAYMENTS_TO_ACQUIRE_PROPERTY_PLANT_AND_EQUIPMENT
from stock_valuation.constants.us_gaap_financial_keys import US_GAAP_ASSETS_CURRENT
from stock_valuation.constants.us_gaap_financial_keys import US_GAAP_LIABILITIES_CURRENT


revenue = RowFinancialsMapper([US_GAAP_REVENUES, US_GAAP_REVENUE_FROM_CONTRACT_WITH_CUSTOMER_EXCLUDING_ASSESSED_TAX])
cost_of_revenue = RowFinancialsMapper([US_GAAP_COST_OF_REVENUE, US_GAAP_COST_OF_GOODS_AND_SERVICE_EXCLUDING_DEPRECIATION_DEPLETION_AND_AMORTIZATION])
current_assets = RowFinancialsMapper([US_GAAP_ASSETS_CURRENT])
current_liabilities = RowFinancialsMapper([US_GAAP_LIABILITIES_CURRENT])
mapping_config = {
    REVENUE: revenue,
    COST_OF_REVENUE: cost_of_revenue,
    OPERATING_EXPENSE: revenue - cost_of_revenue - RowFinancialsMapper([US_GAAP_OPERATING_INCOME_LOSS]),
    DEPRECIATION: RowFinancialsMapper([US_GAAP_DEPRECIATION_DEPLETION_AND_AMORTIZATION]),
    PROVISION_FOR_TAXES: RowFinancialsMapper([US_GAAP_INCOME_TAX_EXPENSE_BENEFIT]),
    CAPITAL_EXPENDITURE: RowFinancialsMapper([US_GAAP_PAYMENTS_TO_ACQUIRE_PROPERTY_PLANT_AND_EQUIPMENT]),
    WORKING_CAPITAL_DELTA: DeltaFinancialsMapper(current_assets - current_liabilities),
}

mapper = CompositeFinancialsMapper(mapping_config)

In [23]:
historical = mapper.map(financials)

In [24]:
historical

,2020,2021,2022,2023,2024
revenue,1.113900e+10,17455000000,31877000000,37281000000,43978000000
cost_of_revenue,5.154000e+09,9351000000,19659000000,22457000000,26651000000
operating_expense,1.084800e+10,11938000000,14050000000,13714000000,14528000000
depreciation,0.000000e+00,0,0,0,-711000000
provision_for_taxes,-1.920000e+08,-492000000,-181000000,-213000000,-5800000000
capital_expenditure,-6.160000e+08,-298000000,-252000000,-223000000,-242000000
working_capital_delta,NaN,-3222000000,601000000,1447000000,-1074000000


In [8]:
# config of forecasting methods -> forecaster -forecast()-> fin_forecast_df
# methods: RecentValue, Average, CAGR, % of <key>, LinReg
# "% of <key>" - is complicated since depends of forecast itself

In [ ]:
class ForecastMethod:
    def forecast(self, df, key):
        pass

class Forecaster:
    def __init__(histrical, config):
        self._historical = historical
        self._config = config

    def forecast(self, period_count=1):
        df = self._historical.copy()
        for _ in range(period_count):
            pass
            # generate next col from config
            for key, method in self._config.items():
                pass
                new_col = method.forecast(df, key)
            # add new col to df
        # cut off historical cols and return the forecast